In [1]:
import numpy as np
import os
import pandas as pd
import json
import pickle

from tqdm.notebook import tqdm

In [2]:
dir = '/content/drive/My Drive/Colab Notebooks/Genshin Impact Monte Carlo/'
os.listdir(dir)

['banners.json',
 'simres_s10k_r210.pkl',
 'simorder_s10k_r210.pkl',
 'simres_s200k_r1000.pkl',
 'simorder_s200k_r1000.pkl',
 'Untitled0.ipynb',
 'genshin_montecarlo.ipynb']

In [3]:
# Load banner data from JSON
jsonpath = dir+'banners.json'

with open(jsonpath, 'r') as j:
  all_banners = json.loads(j.read())

In [41]:
print(all_banners)

{'banner': [{'geninfo': {'name': 'Venti', 'prob5': 0.006, 'prob4': 0.051, 'prob3': 0.943, '5startotal': 6, '4startotal': 29}, '5stardrop_char': ['Venti', 'Keqing', 'Mona', 'Qiqi', 'Diluc', 'Jean'], '4stardrop_char': ['Barbara', 'Fischl', 'Xiangling', 'Sucrose', 'Chongyun', 'Noelle', 'Bennett', 'Ninguang', 'Xingqiu', 'Beidou', 'Razor'], 'rateups': {'5 star': [0], '4 star': [0, 1, 2]}, 'rateup_perc': {'5 star': 0.5, '4 star': 0.5}, 'pitypullnum': {'5 star': 90, '4 star': 10}}]}


In [40]:
# Define the banner class based on JSON data
class Banner():
  def __init__(self, banner):

    self.list_chars = {}

    self.name = banner['geninfo']['name']
    self.prob5 = banner['geninfo']['prob5']
    self.prob4 = banner['geninfo']['prob4']
    self.prob3 = banner['geninfo']['prob3']
    self.total5s = banner['geninfo']['5startotal']
    self.total4s = banner['geninfo']['4startotal']

    self.list_chars['5 star'] = banner['5stardrop_char']
    self.list_chars['4 star'] = banner['4stardrop_char']

    self.rateups = banner['rateups']
    self.rateups_perc = banner['rateup_perc']

    self.pitynum = banner["pitypullnum"]

    self.startup_text()

  def startup_text(self):
    print("Loaded Banner: " + self.name)

    for stars in self.rateups:
      print("Characters on " + stars + " rateup: " + str([self.list_chars[stars][x] for x in self.rateups[stars]]) + " at " + str(self.rateups_perc[stars]*100) + "%")

In [46]:
# Define simulator

class MonteCarloSimulator():
  def __init__(self, Banner, timesrolled=0, nrolls=5, nsims=5):
    self.Banner = Banner
    self.name = self.Banner.name
    self.nrolls = nrolls
    self.nsims = nsims
    self.timesrolled = timesrolled # Number of times already rolled on the banner


    
    self.get4rated = False
    self.get5rated = False

    self.simresults = []
    self.simlist = []

  def _roll1(self)->str:
    roll = ""
    thresh5 = self.Banner.prob5*1000
    thresh4 = self.Banner.prob4*1000
    whichchar = np.random.randint(0, 1000)
    if whichchar <= thresh5:
      roll = self._rollforstar('5 star')
    elif whichchar <= thresh4:
      roll = self._rollforstar('4 star')
    else:
      roll = '3 star'
    return roll

    

  def _rollforstar(self, star)->str:
    # roll = -1

    if self.get5rated:
      whichchar = np.random.randint(0, len(self.Banner.rateups['5 star']))
      self.get5rated = False
    elif self.get4rated:
      whichchar = np.random.randint(0, len(self.Banner.rateups['4 star']))
      self.get4rated = False

    israted = np.random.randint(0, 1000)
    # print(israted)
    if israted < self.Banner.rateups_perc[star]*1000:
      # print('Get rated')
      # Get a rated up char!
      whichchar = np.random.randint(0, len(self.Banner.rateups[star]))
      # rolledchar = self.Banner.total5s[whichchar]
    else:
      # Get an unrated-up char!
      # print('Get unrated')
      whichchar = np.random.randint(0, self.Banner.total4s)
      # print(whichchar)
      while whichchar in self.Banner.rateups[star]:
        whichchar = np.random.randint(0, self.Banner.total4s)
      if whichchar >= len(self.Banner.list_chars[star]):
        return "4 star weapon"
      
    return self.Banner.list_chars[star][whichchar]
      



  def _sim1(self)->(dict, list):

    results = {}
    simorder = []

    simrolled = self.timesrolled

    pity5cnt = self.timesrolled%self.Banner.pitynum['5 star']
    pity4cnt = self.timesrolled%self.Banner.pitynum['4 star']


    if simrolled>0 and ((simrolled+1) % self.Banner.pitynum['5 star'] == 0):
      guarantee5 = True
    if simrolled>0 and ((simrolled+1) % self.Banner.pitynum['4 star'] == 0):
      guarantee4 = True
    else:
      guarantee5, guarantee4 = False, False
    
    for i in range(self.nrolls):
      # print("New Roll! " + str(guarantee5) + " | " + str(guarantee4))
      roll = ""
      if guarantee5:
        # print('i: ' + str(i) + ' 5star pity')
        roll = self._rollforstar('5 star')
        pity5cnt = 0
        guarantee5 = False
      elif guarantee4:
        roll = self._rollforstar('4 star')
        pity4cnt = 0
        guarantee4 = False
      else:        
        roll = self._roll1()
        if roll in self.Banner.list_chars['5 star']:
          # print('Natural 5 stopping pity at: !' + str(pity5cnt))
          pity5cnt = 0
        elif roll in self.Banner.list_chars['4 star']:
          # print('Natural 4 stopping pity at: !' + str(pity4cnt))
          pity4cnt = 0


      # print(roll)

      simorder.append(roll)

      if roll not in results.keys():
        results[roll] = 1
        if roll in self.Banner.list_chars['5 star'] and roll not in self.Banner.rateups['5 star'] and not self.get5rated:
          self.get5rated = True
        elif roll in self.Banner.list_chars['4 star'] and roll not in self.Banner.rateups['4 star'] and not self.get4rated:
          self.get4rated = True
      else:
        results[roll] = results[roll] + 1
      
      simrolled += 1
      pity5cnt += 1
      pity4cnt += 1

      if ((pity5cnt+1) % self.Banner.pitynum['5 star']) == 0:
        # print('pity time 5 ')
        guarantee5 = True
      if ((pity4cnt+1) % self.Banner.pitynum['4 star']) == 0:
        guarantee4 = True

      # print(self.Banner.pitynum['5 star'])
      # print(pity5cnt, pity5cnt+1%self.Banner.pitynum['5 star'])

    return results, simorder

  def runallsims(self):
    for i in tqdm(range(self.nsims)):
      single_sim, single_simorder = self._sim1()
      self.simresults.append(single_sim)
      self.simlist.append(single_simorder)

  def testroll(self):
    res = {}
    for i in range(10):
      roll = self._rollforstar('4 star')
      if roll not in res.keys():
        res[roll] = 1
      else:
        res[roll] = res[roll] + 1
    print(res)


In [ ]:
# Define a single roll

In [47]:
# Testing
b = Banner(all_banners['banner'][0])
sim = MonteCarloSimulator(b, nrolls=93, nsims=10000)

Loaded Banner: Venti
Characters on 5 star rateup: ['Venti'] at 50.0%
Characters on 4 star rateup: ['Barbara', 'Fischl', 'Xiangling'] at 50.0%


In [48]:
print(sim.Banner.list_chars['5 star'])
sim.testroll()

['Venti', 'Keqing', 'Mona', 'Qiqi', 'Diluc', 'Jean']
{'4 star weapon': 5, 'Noelle': 2, 'Fischl': 1, 'Xiangling': 1, 'Barbara': 1}


In [49]:
sim.runallsims()

In [50]:
# Turn sims into a pandas dataframe

simdf = pd.DataFrame(sim.simresults)
simdf = simdf.fillna(0)
simdf['5 star'] = simdf[sim.Banner.list_chars['5 star']].sum(axis=1)
simdf['4 star'] = simdf[sim.Banner.list_chars['4 star']].sum(axis=1)
simdf['Total Pulls'] = simdf[simdf.columns[:-2]].sum(axis=1)
simdf

,4 star weapon,3 star,Barbara,Fischl,Beidou,Xiangling,Qiqi,Razor,Sucrose,Xingqiu,Bennett,Venti,Chongyun,Ninguang,Noelle,Jean,Diluc,Mona,Keqing,5 star,4 star,Total Pulls
0,6.0,77,4.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,9.0,93.0
1,6.0,79,3.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,93.0
2,5.0,81,2.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,93.0
3,7.0,79,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,93.0
4,6.0,80,2.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,93.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,5.0,81,0.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.0,93.0
9996,4.0,80,3.0,1.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,93.0
9997,6.0,79,4.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,7.0,93.0
9998,7.0,80,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,93.0


In [53]:
# Determine the condition
# simslice = simdf[simdf['5 star'] > 6]
# simslice = simdf[(simdf['Barbara'] >= 6) & (simdf['Fischl'] >= 7) & (simdf['Keqing'] >= 1) & (simdf['Diluc'] >= 1) & (simdf['Qiqi'] >= 1) & (simdf['Mona'] >= 1) & (simdf['Jean'] == 0) & (simdf['4 star'] >= 39)  & (simdf['Venti'] >= 7)]
simslice = simdf[(simdf['Barbara'] >= 4) & (simdf['Fischl'] >= 5) & (simdf['Venti'] == 0)]
# simslice = simdf[(simdf['Venti'] >= 1) & (simdf['4 star'] >= 3 )]
# simslice = simdf[(simdf['Venti'] == 0) & (simdf['5 star'] >= 2 )]
# simslice = simdf[(simdf['4 star'] >= 4) & (simdf['Razor'] >= 1)]
simslice

,4 star weapon,3 star,Barbara,Fischl,Beidou,Xiangling,Qiqi,Razor,Sucrose,Xingqiu,Bennett,Venti,Chongyun,Ninguang,Noelle,Jean,Diluc,Mona,Keqing,5 star,4 star,Total Pulls
4515,3.0,77,4.0,7.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,93.0
4667,2.0,79,4.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,93.0
7726,1.0,78,5.0,5.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,1.0,13.0,93.0
8948,4.0,80,4.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,93.0


In [54]:
# Analyze and get data

print(simdf.mean())
print(simdf.std())
print(str((len(simslice.index)/sim.nsims)*100) + "%")



4 star weapon     4.8217
3 star           79.2653
Barbara           2.0561
Fischl            2.0899
Beidou            0.2393
Xiangling         2.0701
Qiqi              0.0229
Razor             0.2412
Sucrose           0.2397
Xingqiu           0.2456
Bennett           0.2359
Venti             0.6609
Chongyun          0.2450
Ninguang          0.2370
Noelle            0.2365
Jean              0.0221
Diluc             0.0246
Mona              0.0229
Keqing            0.0233
5 star            0.7767
4 star            8.1363
Total Pulls      93.0000
dtype: float64
4 star weapon    1.948150
3 star           1.587378
Barbara          1.313440
Fischl           1.333265
Beidou           0.486271
Xiangling        1.326183
Qiqi             0.150259
Razor            0.490761
Sucrose          0.482563
Xingqiu          0.490413
Bennett          0.488339
Venti            0.577879
Chongyun         0.493558
Ninguang         0.487293
Noelle           0.481654
Jean             0.148370
Diluc            0.

In [ ]:
filename_extension = '_s' + str(int(sim.nsims/1000)) + 'k_r' + str(sim.nrolls)

with open(dir+'simres'+filename_extension + '.pkl', 'wb') as f:
  pickle.dump(sim.simresults, f)

with open(dir+'simorder'+filename_extension+'.pkl', 'wb') as f:
  pickle.dump(sim.simlist, f)



In [ ]:
os.listdir(dir)

['genshin_montecarlo.ipynb',
 'banners.json',
 'simres_s10k_r210.pkl',
 'simorder_s10k_r210.pkl',
 'simres_s200k_r1000.pkl',
 'simorder_s200k_r1000.pkl']